In [3]:
#Libraries
import re

### This code is adapted from Baltic, a python package written by Gytis Dudas, for parsing BEAST trees files.
Need to:
    count number of tax (ntax)
    have tip translation dictionary

In [53]:
with open('/Users/alliblk/Desktop/gitrepos/zika-usvi/scripts/alignment_americas_monophyly.trees','rU') as file:
    taxaTranslation = False
    
    for line in file: ## iterate through each line
    ###################################################################################
        if 'state' not in line.lower(): #going to grab all the interesting stuff in the .trees file prior to the newick tree strings
            matchTaxonCount = re.search('Dimensions ntax\=([0-9]+)\;',line) ## Extract useful information from the bits preceding the actual trees.
            if matchTaxonCount is not None:
                nTaxa = int(matchTaxonCount.group(1))
        
        if 'Translate' in line: #int to full taxon name mapping follows in file
            taxaTranslation = True
            tipNameMap = {}
        
        if taxaTranslation == True:
            matchMapping = re.search('([0-9]+) ([\'\"A-Za-z0-9\?\|\-\_\.\/]+)',line)
            if matchMapping is not None:
                tipNameMap[matchMapping.group(1)] = matchMapping.group(2)
        
        if 'tree STATE_' in line: #sampled tree strings now
            crudMatch = re.match('tree\sSTATE\_([0-9]+).+\[\&R\]\s',line)
            treeStringMatch = #NEED TO MAKE TREE STRING MATCH
            print crudMatch.group(0)
            break
    

    #Checks
    assert len(tipNameMap) == nTaxa, 'not all tips read in by regex'
                


tree STATE_0 [&lnP=-70336.1258368917,posterior=-70336.1258368917] = [&R] 
